In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import pickle
app = Flask(__name__)

# Load the dataset
df_list = pd.read_html('/home/fares/DatasetsOrange/offer.xls')
df = df_list[0]
df['Period'] = pd.to_datetime(df['Period'])  # Convert the index column to Timestamp objects
df = df.set_index('Period')
# Preprocess the dataset
float_cols = df.columns
df[float_cols] = df[float_cols].applymap(lambda x: str(x).replace(",", "."))
df[float_cols] = df[float_cols].applymap(lambda x: str(x).replace(" ", ""))
df[float_cols] = df[float_cols].apply(pd.to_numeric, errors='coerce', axis=1)
df[float_cols] = df[float_cols].applymap(lambda x: x / (1024 * 1024 * 1024))

# Model file path
model_path = 'ARIMA.pkl'

# Load the ARIMA model
with open(model_path, 'rb') as file:
    model_fit = pickle.load(file)


@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Get JSON data from the request

    column_name = data['column_name']
    desired_days = data['desired_days']

    # Generate a range of dates for prediction
    start_date = df.index[-1] + pd.DateOffset(days=1)
    end_date = start_date + pd.DateOffset(days=desired_days)

    # Convert start_date and end_date to UTC timestamps
    start_date_utc = start_date.tz_localize(None)
    end_date_utc = end_date.tz_localize(None)

    # Generate predictions using the loaded ARIMA model
    predictions = model_fit.predict(start=start_date_utc, end=end_date_utc)

    # Generate prediction dates in the local timezone
    prediction_dates_local = pd.date_range(start=start_date, end=end_date, tz=df.index.tz)

    # Create a list of dictionaries with dates and predictions
    predictions_with_dates = [
        {'date': date.strftime('%Y-%m-%d %H:%M:%S'), 'prediction': prediction}
        for date, prediction in zip(prediction_dates_local, predictions)
    ]

    # Create response JSON
    response = {
        'predictions': predictions_with_dates
    }

    # Generate the plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df[column_name], label='Real Values')
    ax.plot(prediction_dates_local, predictions, label='Predictions')
    ax.set_xlabel('Period')
    ax.set_ylabel('Values')
    ax.set_title('{} Predictions'.format(column_name))  # Set the title dynamically
    ax.legend()

    # Save the plot image to a buffer
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)

    # Encode the plot image to base64
    plot_image_data = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response['plot_image'] = plot_image_data

    return jsonify(response)


@app.route('/descriptive-curve', methods=['POST'])
def descriptive_curve():
    
    try:
        data = request.get_json()

        # Get input parameters from the request
        column_name = data['column_name']
        start_date = pd.to_datetime(data['start_date'])
        end_date = pd.to_datetime(data['end_date'])

        # Subset the dataset based on the specified date range
        subset_df = df.loc[start_date:end_date, column_name]

        # Increase the figure size
        plt.figure(figsize=(15, 6))

        # Plot the descriptive curve
        plt.plot(subset_df.index, subset_df.values)
        plt.xlabel('Period')
        plt.ylabel('Values')
        plt.title('Descriptive Curve of {} in GB'.format(column_name))
        plt.legend([column_name])

        # Save the plot image to a buffer
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)

        # Encode the plot image to base64
        plot_image_data = base64.b64encode(buffer.getvalue()).decode('utf-8')

        # Create the response
        response = {
            'plot_image': plot_image_data
        }

        return jsonify(response)

    except Exception as e:
        error_message = str(e)
        response = {
            'error': error_message
        }
        return jsonify(response), 500



if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'